# Imports

In [1]:
import logging
import shutil
import sys
from os.path import isdir, join
from tempfile import gettempdir
from uuid import uuid4

import folium
import pandas as pd
from aequilibrae import Parameters, Project, logger

import numpy as np
from tabulate import tabulate
import geopandas as gpd

# Model place (Country) to build the model for

In [2]:
model_place = 'Andorra'

In [3]:
fldr = f'/home/jovyan/workspace/road_analytics/{model_place}'

if isdir(fldr):
    print(fldr)

/home/jovyan/workspace/road_analytics/Andorra


# Open Model

In [4]:
project = Project()
project.open(fldr)

# Get model data

In [5]:
links = pd.read_sql('select * from links', project.conn).drop(columns=['geometry'])

In [6]:
links.head().transpose()

,0,1,2,3,4
ogc_fid,1,2,3,4,5
link_id,1,2,3,4,5
a_node,2,1,5,4,8
b_node,1,3,4,6,7
direction,1,1,1,1,1
distance,19.863131,16.884412,15.190793,18.038848,13.234459
modes,ct,ct,ct,ct,ct
link_type,residential,residential,residential,residential,residential
name,None,None,None,None,None
speed_ab,NaN,NaN,NaN,NaN,NaN


In [7]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9448 entries, 0 to 9447
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ogc_fid      9448 non-null   int64  
 1   link_id      9448 non-null   int64  
 2   a_node       9448 non-null   int64  
 3   b_node       9448 non-null   int64  
 4   direction    9448 non-null   int64  
 5   distance     9448 non-null   float64
 6   modes        9448 non-null   object 
 7   link_type    9448 non-null   object 
 8   name         2936 non-null   object 
 9   speed_ab     1387 non-null   float64
 10  speed_ba     1084 non-null   float64
 11  capacity_ab  0 non-null      object 
 12  capacity_ba  0 non-null      object 
 13  osm_id       9448 non-null   int64  
 14  bridge       440 non-null    object 
 15  tunnel       76 non-null     object 
 16  toll         15 non-null     object 
 17  surface      3506 non-null   object 
 18  lanes_ab     1755 non-null   float64
 19  lanes_

---
Useful analytics:

* Surface type (pavement);
* Num. of nodes (per mode)
* Node density (per km²)
* Length of links (per mode)
* Density of links (per km²)
* Num.tunnels
* Num. nodes corresponding to tunnels
* Length of tunnels
* Num. tunnels
* Num. nodes corresponding to tunnels
* Length of tunnels
* Num. tunnels
* Num. nodes corresponding to tunnel roads
* Length of tunnel roads
* Avg. Speed
* Avg. num. of lanes
---

nodes = pd.read_sql('select * from nodes', project.conn).drop(columns=['geometry'])

nodes.head()

In [36]:
 bridge_links = links[links["bridge"].notna()]

0

In [40]:
def bridge_stats(links):
    #vehicle_links[vehicle_links['bridge'].notna()][['a_node', 'b_node']].nunique().sum()
    vehicle_links = links[(links["modes"].str.contains("c")) & (links["modes"].str.contains("t"))]
    bridge_links = vehicle_links[vehicle_links["bridge"].notna()]
    
    print("----- Overall stats -----")
    print(" ")
    table = [["bridges", len(bridge_links), bridge_links[['a_node', 'b_node']].nunique().sum(),
              np.around(bridge_links.sum(numeric_only=True)['distance'], decimals=2)],
             ["Total", len(vehicle_links), vehicle_links[['a_node', 'b_node']].nunique().sum(),
              np.around(vehicle_links.sum(numeric_only=True)['distance'], decimals=2)]]

    print(tabulate(table, headers=["", "Links", "Nodes", "km"], tablefmt='github', floatfmt=".2f"))
    print(" ")
    if len(bridge_links[bridge_links['toll'].notna()]) == 0:
        print(f"There are no toll bridges in {model_place}.")
        print("")
    else: #gotta work on this one later!
        is_bridge(bridge_links)
    
    print("----- Link types -----")
    print("")
    df = pd.DataFrame.from_dict(dict(bridge_links.groupby('link_type').sum(numeric_only=True)['distance']/1000),
                            orient='index', columns=['total_km'])
    df.insert(0, 'links', bridge_links.groupby('link_type').count()['distance'])
    df.insert(1, 'nodes', [bridge_links[bridge_links['link_type'] == i][['a_node', 'b_node']].nunique().sum() 
                           for i in bridge_links.groupby('link_type').count()['distance'].index.tolist()])

    print(tabulate(df, tablefmt="github", floatfmt=(".0f", ".0f", ".0f", ".2f"), 
                   headers=["Link Type", "Links", "Nodes", 'Total km']))
    print("")
    
    print("----- Pavement Surfaces -----")
    print("")
    bridge_links.loc[bridge_links['surface'].isna(), 'surface'] = 'unclassified'
    df = pd.DataFrame.from_dict(dict(bridge_links.groupby('surface').sum(numeric_only=True)['distance']/1000),
                            orient='index', columns=['total_km'])
    df.insert(0, 'links', bridge_links.groupby('surface').count()['distance'])
    df.insert(1, 'nodes', [bridge_links[bridge_links['surface'] == i][['a_node', 'b_node']].nunique().sum() 
                           for i in bridge_links.groupby('surface').count()['distance'].index.tolist()])

    print(tabulate(df, tablefmt="github", floatfmt=(".0f", ".0f", ".0f", ".2f"), 
                   headers=["Surface", "Links", "Nodes", 'Total km']))
    print("")

In [34]:
def is_bridge(bridge_links):
    
    print(f"----- Toll bridges -----")
    print(" ")
    bridge_bridges = bridge_links[bridge_links['toll'] == 'yes']
    table = [[len(bridge_bridges),
              bridge_bridges[['a_node', 'b_node']].nunique().sum(),
              bridge_bridges.sum(numeric_only=True)['distance']/1000]]
    
    print(tabulate(table, headers=['links', 'nodes','total_km'], tablefmt='github', floatfmt=".2f"))
    print("")

In [41]:
bridge_stats(links)

----- Overall stats -----
 
|         |   Links |   Nodes |        km |
|---------|---------|---------|-----------|
| bridges |     288 |  525.00 |   7536.45 |
| Total   |    5007 | 7847.00 | 506772.07 |
 
There are no toll bridges in Andorra.

----- Link types -----

| Link Type     |   Links |   Nodes |   Total km |
|---------------|---------|---------|------------|
| living_street |       1 |       2 |       0.01 |
| primary       |     106 |     189 |       3.42 |
| residential   |      57 |     113 |       0.89 |
| secondary     |      57 |     102 |       1.21 |
| service       |      29 |      58 |       0.54 |
| tertiary      |      11 |      22 |       0.11 |
| trunk         |      12 |      21 |       1.06 |
| unclassified  |      15 |      30 |       0.29 |

----- Pavement Surfaces -----

| Surface      |   Links |   Nodes |   Total km |
|--------------|---------|---------|------------|
| asphalt      |     185 |     334 |       5.27 |
| unclassified |     103 |     204 |   

In [23]:
def toll_stats(links):
    
    vehicle_links = links[(links["modes"].str.contains("c")) & (links["modes"].str.contains("t"))]
    toll_links = vehicle_links[vehicle_links["toll"].notna()]
    
    print("----- Overall stats -----")
    print(" ")
    table = [["Tolls", len(toll_links), toll_links[['a_node', 'b_node']].nunique().sum(),
              np.around(toll_links.sum(numeric_only=True)['distance'], decimals=2)],
             ["Total", len(vehicle_links), vehicle_links[['a_node', 'b_node']].nunique().sum(),
              np.around(vehicle_links.sum(numeric_only=True)['distance'], decimals=2)]]

    print(tabulate(table, headers=["", "Links", "Nodes", "km"], tablefmt='github', floatfmt=".2f"))
    print(" ")
    if len(toll_links.toll.value_counts()) == 0:
        print(f"There are no tolls in {model_place}.")
        print("")
    else: #gotta work on this one later!
        is_toll(toll_links)
    
    print("----- Link types -----")
    print("")
    df = pd.DataFrame.from_dict(dict(toll_links.groupby('link_type').sum(numeric_only=True)['distance']/1000),
                            orient='index', columns=['total_km'])
    df.insert(0, 'links', toll_links.groupby('link_type').count()['distance'])
    df.insert(1, 'nodes', [toll_links[toll_links['link_type'] == i][['a_node', 'b_node']].nunique().sum() 
                           for i in toll_links.groupby('link_type').count()['distance'].index.tolist()])

    print(tabulate(df, tablefmt="github", floatfmt=(".0f", ".0f", ".0f", ".2f"), 
                   headers=["Link Type", "Links", "Nodes", 'Total km']))
    print("")
    
    print("----- Pavement Surfaces -----")
    print("")
    toll_links.loc[toll_links['surface'].isna(), 'surface'] = 'unclassified'
    df = pd.DataFrame.from_dict(dict(toll_links.groupby('surface').sum(numeric_only=True)['distance']/1000),
                            orient='index', columns=['total_km'])
    df.insert(0, 'links', toll_links.groupby('surface').count()['distance'])
    df.insert(1, 'nodes', [toll_links[toll_links['surface'] == i][['a_node', 'b_node']].nunique().sum() 
                           for i in toll_links.groupby('surface').count()['distance'].index.tolist()])

    print(tabulate(df, tablefmt="github", floatfmt=(".0f", ".0f", ".0f", ".2f"), 
                   headers=["Surface", "Links", "Nodes", 'Total km']))
    print("")

In [24]:
def is_toll(toll_links):
    
    print(f"----- Tolls -----")
    print(" ")
    toll_tolls = toll_links[toll_links['toll'] == 'yes']
    table = [[len(toll_tolls),
              toll_tolls[['a_node', 'b_node']].nunique().sum(),
              toll_tolls.sum(numeric_only=True)['distance']/1000]]
    
    print(tabulate(table, headers=['links', 'nodes','total_km'], tablefmt='github', floatfmt=".2f"))
    print("")

In [25]:
toll_stats(links)

----- Overall stats -----
 
|       |   Links |   Nodes |        km |
|-------|---------|---------|-----------|
| Tolls |      15 |   27.00 |   4489.18 |
| Total |    5007 | 7847.00 | 506772.07 |
 
----- Tolls -----
 
|   links |   nodes |   total_km |
|---------|---------|------------|
|      13 |   23.00 |       4.27 |

----- Link types -----

| Link Type   |   Links |   Nodes |   Total km |
|-------------|---------|---------|------------|
| primary     |       2 |       4 |       0.22 |
| trunk       |      13 |      23 |       4.27 |

----- Pavement Surfaces -----

| Surface   |   Links |   Nodes |   Total km |
|-----------|---------|---------|------------|
| asphalt   |      15 |      27 |       4.49 |



In [17]:
def tunnel_stats(links):
    
    vehicle_links = links[(links["modes"].str.contains("c")) & (links["modes"].str.contains("t"))]
    #vehicle_nodes = nodes[nodes['node_id'].isin(vehicle_links.a_node.tolist() + vehicle_links.b_node.tolist())]
    tunnel_links = vehicle_links[vehicle_links["tunnel"].notna()]
    #tunnel_nodes = nodes[nodes['node_id'].isin(tunnel_links.a_node.tolist() + tunnel_links.b_node.tolist())]
    
    print("----- Overall stats -----")
    print(" ")
    table = [["tunnels", len(tunnel_links), tunnel_links[["a_node", 'b_node']].nunique().sum(),
              np.around(tunnel_links.sum(numeric_only=True)['distance']/1000, decimals=2)],
             ["Total", len(vehicle_links), vehicle_links[["a_node", 'b_node']].nunique().sum(),
              np.around(vehicle_links.sum(numeric_only=True)['distance']/1000, decimals=2)]]

    print(tabulate(table, headers=["", "Links", "Nodes", "km"], tablefmt='github', floatfmt=".2f"))
    print(" ")
    
    if len(tunnel_links.tunnel.value_counts()) == 0:
        print(f"There are no toll tunnels in {model_place}.")
        print("")
    else: #gotta work on this one later!
        is_tunnel(tunnel_links)
        
    print("----- Link types -----")
    print("")
    df = pd.DataFrame.from_dict(dict(tunnel_links.groupby('link_type').sum(numeric_only=True)['distance']/1000),
                            orient='index', columns=['total_km'])
    df.insert(0, 'links', tunnel_links.groupby('link_type').count()['distance'])
    df.insert(1, 'nodes', [tunnel_links[tunnel_links['link_type'] == i][['a_node', 'b_node']].nunique().sum() 
                           for i in tunnel_links.groupby('link_type').count()['distance'].index.tolist()])

    print(tabulate(df, tablefmt="github", floatfmt=(".0f", ".0f", ".0f", ".2f"), 
                   headers=["Link Type", "Links", "Nodes", 'Total km']))
    print("")
    
    print("----- Pavement Surfaces -----")
    print("")
    tunnel_links.loc[tunnel_links['surface'].isna(), 'surface'] = 'unclassified'
    df = pd.DataFrame.from_dict(dict(tunnel_links.groupby('surface').sum(numeric_only=True)['distance']/1000),
                            orient='index', columns=['total_km'])
    df.insert(0, 'links', tunnel_links.groupby('surface').count()['distance'])
    df.insert(1, 'nodes', [tunnel_links[tunnel_links['surface'] == i][['a_node', 'b_node']].nunique().sum() 
                           for i in tunnel_links.groupby('surface').count()['distance'].index.tolist()])

    print(tabulate(df, tablefmt="github", floatfmt=(".0f", ".0f", ".0f", ".2f"), 
                   headers=["Surface", "Links", "Nodes", 'Total km']))
    print("")

In [8]:
def is_tunnel(tunnel_links):
    
    print(f"----- Toll tunnels -----")
    print(" ")
    toll_tunnels = tunnel_links[tunnel_links['toll'] == 'yes']
    table = [[len(toll_tunnels),
              toll_tunnels[['a_node', 'b_node']].nunique().sum(),
              toll_tunnels.sum(numeric_only=True)['distance']/1000]]
    
    print(tabulate(table, headers=['links', 'nodes','total_km'], tablefmt='github', floatfmt=".2f"))
    print("")

In [18]:
tunnel_stats(links)

----- Overall stats -----
 
|         |   Links |   Nodes |     km |
|---------|---------|---------|--------|
| tunnels |      63 |  123.00 |  16.55 |
| Total   |    5007 | 7847.00 | 506.77 |
 
----- Toll tunnels -----
 
|   links |   nodes |   total_km |
|---------|---------|------------|
|       2 |    4.00 |       2.96 |

----- Link types -----

| Link Type     |   Links |   Nodes |   Total km |
|---------------|---------|---------|------------|
| living_street |       4 |       8 |       0.03 |
| primary       |      16 |      32 |       3.65 |
| residential   |       4 |       8 |       0.14 |
| secondary     |       2 |       4 |       0.09 |
| service       |      19 |      37 |       3.40 |
| tertiary      |       4 |       8 |       0.37 |
| trunk         |       9 |      18 |       8.81 |
| unclassified  |       5 |       8 |       0.05 |

----- Pavement Surfaces -----

| Link Type    |   Links |   Nodes |   Total km |
|--------------|---------|---------|------------|
| aspha

In [102]:
def primary_stats(links):
    
    vehicle_links = links[(links["modes"].str.contains("c")) & (links["modes"].str.contains("t"))]
    #vehicle_nodes = nodes[nodes['node_id'].isin(vehicle_links.a_node.tolist() + vehicle_links.b_node.tolist())]
    
    print("----- Links -----")
    print("")
    bridge_links = len(vehicle_links[vehicle_links['bridge'].notna()])
    toll_links = len(vehicle_links[vehicle_links['toll'].notna()])
    tunnel_links = len(vehicle_links[vehicle_links['tunnel'].notna()])
    total_links = len(vehicle_links)
    
    table = [[total_links - bridge_links - toll_links - tunnel_links, bridge_links, toll_links, tunnel_links, total_links]]
    
    print(tabulate(table, headers=["General", "Bridge", "Toll", "Tunnel", "Total"], tablefmt="github", floatfmt=".0f"))
    print("")
    
    print("----- Nodes -----")
    print("")
    bridge_nodes = vehicle_links[vehicle_links['bridge'].notna()][['a_node', 'b_node']].nunique().sum()
    toll_nodes = vehicle_links[vehicle_links['toll'].notna()][['a_node', 'b_node']].nunique().sum()
    tunnel_nodes = vehicle_links[vehicle_links['tunnel'].notna()][['a_node', 'b_node']].nunique().sum()
    total_nodes = vehicle_links[['a_node', 'b_node']].nunique().sum()
    
    table = [[total_nodes - bridge_nodes - toll_nodes - tunnel_nodes,
              bridge_nodes, toll_nodes, tunnel_nodes, total_nodes]]
    
    print(tabulate(table, headers=["General", "Bridge", "Toll", "Tunnel", "Total"], tablefmt="github", floatfmt=".0f"))
    print("")
    
    print("----- Total km -----")
    print("")
    bridge_dist = vehicle_links[vehicle_links['bridge'].notna()]['distance'].sum()/1000
    toll_dist = vehicle_links[vehicle_links['toll'].notna()]['distance'].sum()/1000
    tunnel_dist = vehicle_links[vehicle_links['tunnel'].notna()]['distance'].sum()/1000
    total_dist = vehicle_links.distance.sum()/1000
    
    table = [[total_dist - bridge_dist - toll_dist - tunnel_dist,
              bridge_dist, toll_dist, tunnel_dist, total_dist]]
    print(tabulate(table, headers=["General", "Bridge", "Toll", "Tunnel", "Total"], tablefmt="github", floatfmt=".2f"))
    print("")
    
    print("----- Link types -----")
    print("")
    df = pd.DataFrame.from_dict(dict(vehicle_links.groupby('link_type').sum(numeric_only=True)['distance']/1000),
                            orient='index', columns=['total_km'])
    df.insert(0, 'links', vehicle_links.groupby('link_type').count()['distance'])
    df.insert(1, 'nodes', [vehicle_links[vehicle_links['link_type'] == i][['a_node', 'b_node']].nunique().sum() 
                           for i in vehicle_links.groupby('link_type').count()['distance'].index.tolist()])

    print(tabulate(df, tablefmt="github", floatfmt=(".0f", ".0f", ".0f", ".2f"), 
                   headers=["Link Type", "Links", "Nodes", 'Total km']))
    print("")
    
    print("----- Pavement Surfaces -----")
    print("")
    vehicle_links.loc[vehicle_links['surface'].isna(), 'surface'] = 'unclassified'
    df = pd.DataFrame.from_dict(dict(vehicle_links.groupby('surface').sum(numeric_only=True)['distance']/1000),
                            orient='index', columns=['total_km'])
    df.insert(0, 'links', vehicle_links.groupby('surface').count()['distance'])
    df.insert(1, 'nodes', [vehicle_links[vehicle_links['surface'] == i][['a_node', 'b_node']].nunique().sum() 
                           for i in vehicle_links.groupby('surface').count()['distance'].index.tolist()])

    print(tabulate(df, tablefmt="github", floatfmt=(".0f", ".0f", ".0f", ".2f"), 
                   headers=["Surface", "Links", "Nodes", 'Total km']))
    print("")

In [103]:
primary_stats(links)

----- Links -----

|   General |   Bridge |   Toll |   Tunnel |   Total |
|-----------|----------|--------|----------|---------|
|      4641 |      288 |     15 |       63 |    5007 |

----- Nodes -----

|   General |   Bridge |   Toll |   Tunnel |   Total |
|-----------|----------|--------|----------|---------|
|      7172 |      525 |     27 |      123 |    7847 |

----- Total km -----

|   General |   Bridge |   Toll |   Tunnel |   Total |
|-----------|----------|--------|----------|---------|
|    478.20 |     7.54 |   4.49 |    16.55 |  506.77 |

----- Link types -----

| Link Type     |   Links |   Nodes |   Total km |
|---------------|---------|---------|------------|
| living_street |      54 |      97 |       2.02 |
| primary       |    1225 |    2121 |     117.03 |
| residential   |    1449 |    2470 |     114.44 |
| secondary     |     910 |    1681 |     145.93 |
| service       |     856 |    1535 |      57.68 |
| tertiary      |     296 |     567 |      42.80 |
| trunk   

---

In [8]:
sys.path.insert(0, '/home/jovyan/workspace/road_analytics/notebooks')
from functions.country_borders import get_country_borders

In [9]:
# use geopandas area to calculate the country/region/area
place_geo = get_country_borders(model_place)

place_geo.area

---

project.close()